# Notebook for the TF-IDF model

### Import packages

In [1]:
# Import packages
import numpy as np

from pyspark.sql import SparkSession
from pyspark import SQLContext
from pyspark.sql.functions import udf, size, col, countDistinct, collect_list, monotonically_increasing_id, row_number
from pyspark.ml import Pipeline

from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS as gensim_words
import spacy
sp = spacy.load('en_core_web_sm')

import os

from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import Tokenizer, Normalizer, LemmatizerModel, StopWordsCleaner

from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Normalizer as Normalizer_L2

import time

In [2]:
# Import stop words
nltk_stopwords = set(stopwords.words('english')) \
                    .union(set(stopwords.words('german'))) \
                    .union(set(stopwords.words('french')))
gensim_stopwords = set(gensim_words)
spacy_stopwords = sp.Defaults.stop_words
# https://countwordsfree.com/stopwords
cwf_stopwords = set(line.strip() for line in open('stop_words.txt'))

all_stopwords = list( nltk_stopwords \
                        .union(gensim_stopwords) \
                        .union(spacy_stopwords) \
                        .union(cwf_stopwords) )

### Create Spark Context and SQL Context

In [3]:
# Get the right paths on local machine
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ["PYSPARK_PYTHON"] = '/usr/bin/python3.7'
os.environ["PYSPARK_DRIVER_PYTHON"] = '/usr/bin/python3.7'

In [4]:
# Start spark session configured for spark nlp
spark = SparkSession.builder \
        .master('local[*]') \
        .appName('SDDM') \
        .config('spark.driver.memory', '8g') \
        .config('spark.executor.memory', '8g') \
        .config('spark.memory.fraction', '0.8') \
        .config('spark.executor.cores', '8') \
        .config('spark.local.dir', '/home/rikz/Documents/Master/Semester2/SDDM/data/tmp') \
        .config('spark.jars.packages', 'com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.0') \
        .getOrCreate()
print("Created a SparkSession")
sc = spark.sparkContext
print("Created a SparkContext")
sqlContext = SQLContext(sc)
print("Created a SQLContext")

#         .config('spark.local.dir', '/data/s1847503/SDDM/tmp') \

Created a SparkSession
Created a SparkContext
Created a SQLContext


### Load the data into a SQLContext Dataframe

In [5]:
# Load data
df = sqlContext.read.format('csv').options(header='true', maxColumns=2000000) \
        .load('/home/rikz/Documents/Master/Semester2/SDDM/data/data.csv')
#       .load('/data/s1847503/SDDM/newdata/data.csv')

df.show()

+---+--------------------+--------------------+--------------------+--------------------+
|_c0|            paper_id|               title|        list_authors|           full_text|
+---+--------------------+--------------------+--------------------+--------------------+
|  0|           question0|                   -|                   -|How does temperat...|
|  1|           question1|                   -|                   -|Seasonality of tr...|
|  2|           question2|                   -|                   -|Effectiveness of ...|
|  3|           question3|                   -|                   -|Effectiveness of ...|
|  4|           question4|                   -|                   -|Effectiveness of ...|
|  5|           question5|                   -|                   -|Effectiveness of ...|
|  6|           question6|                   -|                   -|Effectiveness of ...|
|  7|           question7|                   -|                   -|Effectiveness of ...|
|  8|     

In [6]:
# Load metadata
df_metadata = sqlContext.read.format('csv').options(header='true') \
                .load('/home/rikz/Documents/Master/Semester2/SDDM/data/metadata.csv') \
                .select(col('sha').alias('paper_id'), 'publish_time', 'title', 'doi', 'journal')

df_metadata.show()

+--------------------+------------+--------------------+--------------------+--------------------+
|            paper_id|publish_time|               title|                 doi|             journal|
+--------------------+------------+--------------------+--------------------+--------------------+
|d1aafb70c066a2068...|  2001-07-04|Clinical features...|10.1186/1471-2334...|      BMC Infect Dis|
|6b0567729c2143a66...|  2000-08-15|Nitric oxide: a p...|        10.1186/rr14|          Respir Res|
|06ced00a5fc042159...|  2000-08-25|Surfactant protei...|        10.1186/rr19|          Respir Res|
|348055649b6b8cf2b...|  2001-02-22|Role of endotheli...|        10.1186/rr44|          Respir Res|
|5f48792a5fa08bed9...|  2001-05-11|Gene expression i...|        10.1186/rr61|          Respir Res|
|b2897e1277f566411...|  2001-12-17|Sequence requirem...|10.1093/emboj/20....|    The EMBO Journal|
|3bb07ea10432f7738...|  2001-03-08|Debate: Transfusi...|       10.1186/cc987|           Crit Care|
|5806726a2

### Initialize Annotators

In [7]:
# Pipeline for text
document_assembler = DocumentAssembler() \
                        .setInputCol('full_text') \
                        .setOutputCol('document')

# Tokenizer divides the text into tokens
tokenizer = Tokenizer() \
                .setInputCols(['document']) \
                .setOutputCol('tokens')

# Finisher converts tokens to human-readable output (we need the tokens for determining the text lengths)
finisher_tokens = Finisher() \
                        .setInputCols(['tokens']) \
                        .setCleanAnnotations(False)

# Normalizer removes punctuation, numbers etc.
normalizer = Normalizer() \
                .setInputCols(['tokens']) \
                .setOutputCol('normalized') \
                .setLowercase(True)

# Lemmatizer changes each word to its lemma
lemmatizer = LemmatizerModel.pretrained() \
                .setInputCols(['normalized']) \
                .setOutputCol('lemma')

# StopWordsCleaner removes stop words    
stopwords_cleaner = StopWordsCleaner() \
                        .setInputCols(['lemma']) \
                        .setOutputCol('clean_lemma') \
                        .setCaseSensitive(False).setStopWords(all_stopwords)

# Finisher converts clean tokens to human-readable output
finisher = Finisher() \
            .setInputCols(['clean_lemma']) \
            .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


### Create Pipeline

In [8]:
# Pipeline for fully preprocessing the text
pipeline = Pipeline() \
            .setStages([
                document_assembler,
                tokenizer,
                normalizer,
                lemmatizer,
                stopwords_cleaner,
                finisher_tokens,
                finisher
             ])

### Preprocess questions

In [9]:
# questions = sqlContext.read.format('csv').options(header='true').load('/data/s1847503/SDDM/newdata/questions.csv')
questions = sqlContext.read.format('csv').options(header='true').load('/home/rikz/Documents/Master/Semester2/SDDM/data/questions.csv')
questions_clean = pipeline.fit(questions).transform(questions)
questions_clean = questions_clean.select('question_id', 'full_text', col('finished_clean_lemma').alias('preprocessed'))
questions_clean.show()

+-----------+--------------------+--------------------+
|question_id|           full_text|        preprocessed|
+-----------+--------------------+--------------------+
|          0|How does temperat...|[temperature, hum...|
|          1|Seasonality of tr...|[seasonality, tra...|
|          2|Effectiveness of ...|[effectiveness, i...|
|          3|Effectiveness of ...|[effectiveness, p...|
|          4|Effectiveness of ...|[effectiveness, s...|
|          5|Effectiveness of ...|[effectiveness, c...|
|          6|Effectiveness of ...|[effectiveness, m...|
|          7|Effectiveness of ...|[effectiveness, c...|
|          8|Significant chang...|[change, transmis...|
|          9|Effectiveness of ...|[effectiveness, w...|
+-----------+--------------------+--------------------+



In [10]:
# Select a question (from 0 to 9)
question_num = 2

questions_clean = questions_clean.filter(questions_clean.question_id == question_num)
q = questions_clean.first().full_text
q

'Effectiveness of inter inner travel restriction'

### Preprocess text

In [11]:
time_before = time.time()

In [12]:
# Peprocess the data
df = pipeline.fit(df).transform(df)
df = df.select('*', size('finished_tokens').alias('text_length'))

df = df.dropna(subset=['paper_id', 'full_text'])
print("Removed empty papers")
df = df.dropDuplicates(subset=['paper_id', 'full_text'])
print("Removed duplicates")
print()

df = df.select(
                col('_c0').alias('id'),
                'paper_id',
                'title',
                'full_text',
                'text_length',
                col('finished_clean_lemma').alias('preprocessed')
            )

df.show()

Before removing empty papers and duplicates: 510 rows.
Removed empty papers
Removed duplicates
After removing empty papers and duplicates: 509 rows.

+---+--------------------+--------------------+--------------------+-----------+--------------------+
| id|            paper_id|               title|           full_text|text_length|        preprocessed|
+---+--------------------+--------------------+--------------------+-----------+--------------------+
| 54|86c3b8562c3094e95...|Title: A Feminist...|"Natural disaster...|        498|[natural, disaste...|
| 56|55743de644f6eda86...|Anti-microbial im...|Chronic obstructi...|       3601|[chronic, obstruc...|
|100|690178e12695c5fdc...|                null|As of this date, ...|       1668|[case, covid, rep...|
| 79|5009e8203ed264e9d...|Coronavirus Rever...|"1 Introduction T...|        316|[introduction, ta...|
|278|7f7940bc0ca1e1986...|Update in the tre...|Each year respira...|       7145|[year, respirator...|
|396|0d76b14e9ae608525...|Viral Li

In [13]:
time_after = time.time()

In [14]:
print('Preprocessing time: {} sec'.format(time_after-time_before) )
# Small dataset: ~15 sec

Preprocessing time: 14.04532241821289 sec


### TF-IDF

In [ ]:
# Compute TF-IDF matrix for papers
tf_p = []
tf_idf_papers = []

tf_p = HashingTF(inputCol='preprocessed', outputCol='tf') \
                    .transform(df)

tf_idf_papers = IDF(inputCol='tf', outputCol='feature') \
                        .fit(tf_p) \
                        .transform(tf_p)

In [ ]:
# Compute TF-IDF matrix for questions
tf_q = []
tf_idf_questions = []

tf_q = HashingTF(inputCol='preprocessed', outputCol='tf') \
                    .transform(questions_clean)

tf_idf_questions = IDF(inputCol='tf', outputCol='feature') \
                        .fit(tf_p) \
                        .transform(tf_q)

In [ ]:
# Delete any extra columns
tf_idf_questions = tf_idf_questions.select('question_id', 'feature')
tf_idf_papers = tf_idf_papers.select('id', 'feature')

In [ ]:
# Compute L2-norm for papers and questions
normalizer_L2 = Normalizer_L2(inputCol='feature', outputCol='norm')
tf_idf_papers = normalizer_L2.transform(tf_idf_papers)
tf_idf_questions = normalizer_L2.transform(tf_idf_questions)

In [ ]:
# Change TF-IDF matrices to block matrices so they can be multiplied
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
matrix_q = IndexedRowMatrix(
                tf_idf_questions \
                    .select('question_id', 'norm') \
                    .rdd.map(lambda row: IndexedRow(row.question_id, row.norm.toArray()))
            ).toBlockMatrix()

matrix_p = IndexedRowMatrix(
                tf_idf_papers \
                    .select('id', 'norm') \
                    .rdd.map(lambda row: IndexedRow(row.id, row.norm.toArray()))
            ).toBlockMatrix()

In [ ]:
# Multiply the matrices to get the cosine similarity matrix
sim_matrix = matrix_p.multiply(matrix_q.transpose())

In [ ]:
# Change the similarity matrix to an array
sim_matrix = sim_matrix.toLocalMatrix().toArray()

In [ ]:
# Change the similarity matrix to a dataframe again to get the most relevant papers for the selected question
relevant = sc.parallelize(sim_matrix[:, question_num].tolist()) \
                .zipWithIndex() \
                .toDF(['similarity', 'id'])

# Remove questions from the paper list
# Sort on cosine similarity
# Take the top 10 relevant documents
relevant = relevant.select('id', 'similarity') \
                .filter(relevant.id > 9) \
                .sort(col('similarity').desc()) \
                .limit(10)

relevant.show()

In [ ]:
# Get the data of the 10 most relevant papers in order of relevance
relevant_ids = [int(row.id) for row in relevant.collect()]
print("Query: {}".format(q))
print()
print("Relevant Papers:")
print()
df_relevant = relevant.join(df.filter(df.id.isin(relevant_ids)), on=['id'], how='left_outer') \
                        .select('paper_id', 'similarity')
df_relevant.show()

In [ ]:
# Create the summary table with the relevant paper from the metadata
df_relevant = df_relevant.join(df_metadata, on=['paper_id'], how='left_outer') \
                            .select('paper_id', 'publish_time', 'title', 'doi', 'journal', 'similarity') \
                            .toPandas() \
                            .sort_values(by='similarity', ascending=False)
df_relevant.head(10)

In [ ]:
# Send the summary table to a csv file
df_relevant.to_csv('/home/rikz/Documents/Master/Semester2/SDDM/SDDM/summary_tables/{}.csv' \
                   .format(q.lower().replace(' ', '_')), index=False)
print("Summary table extracted and sent to csv file.")

### Close Spark Context when done

In [ ]:
sc.stop()